# Clustering Crypto

In [133]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Fetching Cryptocurrency Data

In [134]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"
response = requests.get(url).json()

In [135]:
# Create a DataFrame 
# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.
crypto_df = pd.DataFrame(response['Data']).T
crypto_df.columns

Index(['Id', 'Url', 'ImageUrl', 'ContentCreatedOn', 'Name', 'Symbol',
       'CoinName', 'FullName', 'Description', 'AssetTokenStatus', 'Algorithm',
       'ProofType', 'SortOrder', 'Sponsored', 'Taxonomy', 'Rating',
       'IsTrading', 'TotalCoinsMined', 'CirculatingSupply', 'BlockNumber',
       'NetHashesPerSecond', 'BlockReward', 'BlockTime', 'AssetLaunchDate',
       'MaxSupply', 'MktCapPenalty', 'IsUsedInDefi', 'IsUsedInNft',
       'PlatformType', 'BuiltOn', 'SmartContractAddress', 'DecimalPoints',
       'Difficulty', 'assetWhitepaperUrl'],
      dtype='object')

In [136]:
# Alternatively, use the provided csv file:
from pathlib import Path
file_path = Path("../Data/crypto_data.csv")

# Create a DataFrame
crypto_df_CSV = pd.read_csv(file_path)
crypto_df_CSV.head()

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


### Data Preprocessing

In [137]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
stripped_crypto_df = crypto_df[['CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','MaxSupply']]
stripped_crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,MaxSupply
42,42 Coin,Scrypt,True,PoW/PoS,0,0
300,300 token,N/A,True,N/A,300,300
365,365Coin,X11,True,PoW/PoS,0,0
404,404Coin,Scrypt,True,PoW/PoS,0,0
433,433 Token,N/A,False,N/A,NaN,NaN


In [138]:
# Keep only cryptocurrencies that are trading
stripped_crypto_df = stripped_crypto_df.loc[stripped_crypto_df['IsTrading'] == True]
stripped_crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,MaxSupply
42,42 Coin,Scrypt,True,PoW/PoS,0,0
300,300 token,N/A,True,N/A,300,300
365,365Coin,X11,True,PoW/PoS,0,0
404,404Coin,Scrypt,True,PoW/PoS,0,0
611,SixEleven,SHA-256,True,PoW,0,0


In [139]:
# Keep only cryptocurrencies with a working algorithm
stripped_crypto_df = stripped_crypto_df[stripped_crypto_df['Algorithm'] != 'N/A']
stripped_crypto_df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,MaxSupply
42,42 Coin,Scrypt,True,PoW/PoS,0,0
365,365Coin,X11,True,PoW/PoS,0,0
404,404Coin,Scrypt,True,PoW/PoS,0,0
611,SixEleven,SHA-256,True,PoW,0,0
808,808,SHA-256,True,PoW/PoS,0,0
...,...,...,...,...,...,...
BUNNY,Pancake Bunny,BEP-20 Token,True,N/A,1.11995e+07,-1
YAYCOIN,YAYcoin,Scrypt,True,PoW/PoS,NaN,NaN
ACTIN,Actinium,Lyra2Z,True,PoW,NaN,NaN
RDD,Reddcoin,Scrypt,True,PoW/PoS,3.04109e+10,-1


In [140]:
# Remove the "IsTrading" column
stripped_crypto_df = stripped_crypto_df.drop('IsTrading', axis=1)
stripped_crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,MaxSupply
42,42 Coin,Scrypt,PoW/PoS,0,0
365,365Coin,X11,PoW/PoS,0,0
404,404Coin,Scrypt,PoW/PoS,0,0
611,SixEleven,SHA-256,PoW,0,0
808,808,SHA-256,PoW/PoS,0,0


In [141]:
# Remove rows with at least 1 null value
stripped_crypto_df = stripped_crypto_df.dropna()
stripped_crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,MaxSupply
42,42 Coin,Scrypt,PoW/PoS,0,0
365,365Coin,X11,PoW/PoS,0,0
404,404Coin,Scrypt,PoW/PoS,0,0
611,SixEleven,SHA-256,PoW,0,0
808,808,SHA-256,PoW/PoS,0,0


In [142]:
# Remove rows with cryptocurrencies having no coins mined
stripped_crypto_df = stripped_crypto_df.loc[stripped_crypto_df['TotalCoinsMined'] > 0]
stripped_crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,MaxSupply
NVC,NovaCoin,Scrypt,PoW/PoS,3.37936e+06,-1
NSR,NuShares,PoS,PoS,6.1634e+09,0
MONA,MonaCoin,Scrypt,PoW,8.29052e+07,-1
TRI,Triangles Coin,X13,PoW/PoS,185109,0
SAFEX,SafeExchangeCoin,Scrypt,PoC,2147483647,-1


In [143]:
# Drop rows where there are 'N/A' text values
stripped_crypto_df = stripped_crypto_df.drop(stripped_crypto_df[stripped_crypto_df.values == 'N/A'].index)
stripped_crypto_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,MaxSupply
NVC,NovaCoin,Scrypt,PoW/PoS,3.37936e+06,-1
NSR,NuShares,PoS,PoS,6.1634e+09,0
MONA,MonaCoin,Scrypt,PoW,8.29052e+07,-1
TRI,Triangles Coin,X13,PoW/PoS,185109,0
SAFEX,SafeExchangeCoin,Scrypt,PoC,2147483647,-1
...,...,...,...,...,...
AION,Aion,"Equihash210,9",PoW/PoS,495215020,-1
ACT,Achain,DPoS,DPoS,1000000000,0
ETC,Ethereum Classic,EtcHash,PoW,1.281e+08,210700000
RDD,Reddcoin,Scrypt,PoW/PoS,3.04109e+10,-1


In [144]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
coin_name_df = stripped_crypto_df['CoinName'].to_frame()
coin_name_df

,CoinName
NVC,NovaCoin
NSR,NuShares
MONA,MonaCoin
TRI,Triangles Coin
SAFEX,SafeExchangeCoin
...,...
AION,Aion
ACT,Achain
ETC,Ethereum Classic
RDD,Reddcoin


In [145]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
stripped_crypto_df = stripped_crypto_df.drop('CoinName', axis=1)
stripped_crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,MaxSupply
NVC,Scrypt,PoW/PoS,3.37936e+06,-1
NSR,PoS,PoS,6.1634e+09,0
MONA,Scrypt,PoW,8.29052e+07,-1
TRI,X13,PoW/PoS,185109,0
SAFEX,Scrypt,PoC,2147483647,-1


In [146]:
# Create dummy variables for text features
dummy_crypto_df = pd.get_dummies(stripped_crypto_df)

In [147]:
# Standardize data
scaled_crypto_df = StandardScaler().fit_transform(dummy_crypto_df)

### Reducing Dimensions Using PCA

In [148]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)

In [149]:
# Create a DataFrame with the principal components data
pca_crypto_df = pd.DataFrame(pca.fit_transform(scaled_crypto_df), columns=['PC1', 'PC2', 'PC3'])

### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [150]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    kmeans = KMeans(n_clusters=i, random_state=42)
    kmeans.fit(pca_crypto_df)
    inertia.append(kmeans.inertia_)

# Create the Elbow Curve using hvPlot
data = {'k': k, 'inertia': inertia}
elbow_df = pd.DataFrame(data)
elbow_df.hvplot.line(x='k', y='inertia', xticks=k)

C:\Users\ayres\miniconda3\envs\machineLearning\lib\site-packages\sklearn\cluster\_kmeans.py:882: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.



:Curve   [k]   (inertia)

Running K-Means with `k=<your best value for k here>`

In [151]:
# Initialize the K-Means model
k_model = KMeans(n_clusters=5, random_state=42)
# Fit the model
k_model.fit(pca_crypto_df)
# Predict clusters
predictions = k_model.predict(pca_crypto_df)
# Create a new DataFrame including predicted clusters and cryptocurrencies features
pca_crypto_df['target'] = k_model.labels_
pca_crypto_df['predictions'] = predictions

In [152]:
pca_crypto_df

,PC1,PC2,PC3,target,predictions
0,0.223998,2.177598e-14,6.274356e-15,0,0
1,0.558602,2.666237e-14,9.325725e-15,0,0
2,-0.561728,-1.464449e-14,3.258555e-15,4,4
3,0.419167,1.636906e-14,2.130437e-14,0,0
4,0.160354,2.140474e-14,1.001592e-16,0,0
...,...,...,...,...,...
118,0.466450,3.226116e-14,8.640277e-15,0,0
119,0.629063,3.273681e-14,1.379630e-14,0,0
120,-1.515985,-7.060923e-14,-1.526466e-14,4,4
121,0.223998,2.129505e-14,2.460965e-15,0,0


### Visualizing Results

#### 3D-Scatter with Clusters

In [153]:
# Create a 3D-Scatter with the PCA data and the clusters
threeDfig = px.scatter_3d(
    pca_crypto_df,
    x='PC1',
    y='PC2',
    z='PC3',
    color='target',
    symbol='target',
    width=1000
)
threeDfig.update_layout(legend=dict(x=0, y=1))
threeDfig.show()

#### Table of Tradable Cryptocurrencies

In [162]:
# Table with tradable cryptos
table_crypto_df = pd.DataFrame(stripped_crypto_df)
table_crypto_df['CoinName'] = coin_name_df['CoinName']
table_crypto_df['target'] = pca_crypto_df['target'].values
table_crypto_df.hvplot.table(columns=["CoinName", "Algorithm", "ProofType", "MaxSupply", "TotalCoinsMined", "target"], sortable=True, selectable=True)

:Table   [CoinName,Algorithm,ProofType,MaxSupply,TotalCoinsMined,target]

In [163]:
# Print the total number of tradable cryptocurrencies
print(f"Total Amount of Tradable CryptoCurrencies: {table_crypto_df.shape[0]}")

Total Amount of Tradable CryptoCurrencies: 123


#### Scatter Plot with Tradable Cryptocurrencies

In [167]:
# Scale data to create the scatter plot
mm_scaler = MinMaxScaler()
scatter_crypto = mm_scaler.fit_transform(table_crypto_df[['TotalCoinsMined', 'MaxSupply']].copy())

In [170]:
scatter_crypto_df = pd.DataFrame(scatter_crypto, index=table_crypto_df['CoinName'], columns=['TotalCoinsMined', 'MaxSupply'])
scatter_crypto_df

,TotalCoinsMined,MaxSupply
CoinName,,
NovaCoin,3.377280e-06,0.000000e+00
NuShares,6.163293e-03,9.999833e-13
MonaCoin,8.290184e-05,0.000000e+00
Triangles Coin,1.830845e-07,9.999833e-13
SafeExchangeCoin,2.147446e-03,0.000000e+00
...,...,...
Aion,4.952047e-04,0.000000e+00
Achain,9.999813e-04,9.999833e-13
Ethereum Classic,1.280954e-04,2.106965e-04


In [175]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
scatter_crypto_df.hvplot.scatter(
    x='TotalCoinsMined',
    y='MaxSupply'
)

:Scatter   [TotalCoinsMined]   (MaxSupply)